## Create and Save Data Table 

- we'll want to put all this as a function instead 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import glob
import pickle
import scipy
import statsmodels.api as sm
from datetime import datetime
import os
%matplotlib inline

In [ ]:
import sys
sys.path.append("../functions/")

In [3]:
# data processing functions # 
import Scripts_Data_Processing
reload(Scripts_Data_Processing)
from Scripts_Data_Processing import *

In [50]:
import calculate_questionnaire_scores
reload(calculate_questionnaire_scores)
from calculate_questionnaire_scores import *

In [51]:
files = ['../../questionnaires/'+f for f in os.listdir('../../questionnaires/')] 
files.remove('../../questionnaires/.DS_Store')
files[1:10]

['../../questionnaires/hit_batch_2017_4_17_ambiASIR_A12HZGOZQD5YK7.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A12K9210P1R1DN.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A14ADQ7RUN6TDY.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A14TOBAPF6MMB5.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A18BORMKPR29F2.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A18RE2B07K9M88.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A18TWORM22TW9U.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A1BSA7FVVJGZ15.csv',
 '../../questionnaires/hit_batch_2017_4_17_ambiASIR_A1BWR5RBQDZQ16.csv']

- only using the combined data (other subjects are in a different questionnaires folder)

### Start the table with Questionnaires Scores

In [52]:
def start_data_table_with_questionnaires(files):
    data_participants = pd.DataFrame(columns=['MID','AID','date'])
    from datetime import datetime

    for ff,f in enumerate(files): 
            q = pd.read_csv(f)

            if not q.empty:
                AID = q.AID[0]
                MID = q.MID[0]
                date=str(get_date_from_filename(f))
                if type(MID) is not str:
                    MID=str(MID)
                if type(AID) is not str:
                    AID=str(AID)

                # if MID and date are not already in the dataframe append
                if data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),].empty:
                    row = pd.DataFrame([(MID,AID,date)],columns=['MID','AID','date'])
                    data_participants=data_participants.append(row)


                # fix q to just have mathcing date entries
                q['date'] = q['start_date'].apply(lambda x: str(x.split(' ')[0])) # change the date in q to be just day, month year
                lastdate = q['date'].as_matrix()[-1]
                q = q.loc[q['date']==lastdate] # take just end of the file, some subjects have earlier dates

                date_in_f = datetime.strptime(date,'%Y-%m-%d')
                try:
                    date_in_q = datetime.strptime(q['date'].as_matrix()[0],'%m/%d/%y')
                except:
                    date_in_q = datetime.strptime(q['date'].as_matrix()[0],'%Y-%m-%d')


                if (date_in_f.month==date_in_q.month) and (np.abs(date_in_f.day-date_in_q.day)<10): # make sure the dates in q are within 10 days of the filename


                    # drop duplicate questions
                    length_before = len(q)
                    q = q.drop_duplicates(subset=['question'],keep='last')
                    length_after=len(q)
                    if length_before!=length_after:
                        had_to_drop=1
                    else:
                        had_to_drop=0

                    unique_responses = len(q.answer.value_counts())

                    if 'STAI_Trait' in f:

                        (STAI_Trait,STAI_Trait_anx,STAI_Trait_dep)=calc_stais(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_Trait']=STAI_Trait
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_Trait_anx']=STAI_Trait_anx
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_Trait_dep']=STAI_Trait_dep
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_Trait_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_Trait_unique_responses']=unique_responses

                    if 'STAI_State' in f:
                        (STAI_State)=calc_stais_state(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_State']=STAI_State
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_State_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'STAI_State_unique_responses']=unique_responses

                    if 'BDI' in f:
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'BDI']=calc_bdi(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'BDI_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'BDI_unique_responses']=unique_responses

                    if 'MASQ' in f:

                        (masq_mix_symp,masq_aa,masq_ad,masq_anx_symp,masq_dep_symp)=calc_MASQ(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ.AA']=masq_aa
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ.AD']=masq_ad
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ.MS']=masq_mix_symp
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ.DS']=masq_dep_symp
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ.AS']=masq_anx_symp
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'MASQ_unique_responses']=unique_responses

                    if 'EPQ' in f:
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'EPQ']=calc_EPQ(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'EPQ_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'EPQ_unique_responses']=unique_responses

                    if 'PSWQ' in f:
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'PSWQ']=calc_PSWQ(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'PSWQ_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'PSWQ_unique_responses']=unique_responses

                    if 'CESD' in f:
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'CESD']=calc_CESD(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'CESD_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'CESD_unique_responses']=unique_responses
                    if 'ASIR' in f:
                        (asir,asir16)=calc_ASIR(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'ASIR']=asir
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'ASIR_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'ASIR_unique_responses']=unique_responses

                    if 'IUS' in f:
                        (IUS,IUS1,IUS2)=calc_IUS(q)
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'IUS']=IUS
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'IUS1']=IUS1
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'IUS2']=IUS2
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'IUS_had_to_drop_rows']=had_to_drop
                        data_participants.loc[(data_participants.MID==MID) & (data_participants.date==date),'IUS_unique_responses']=unique_responses
                else:
                    print('bad: '+f)
    return(data_participants)

In [54]:
data_participants = start_data_table_with_questionnaires(files)

bad: ../../questionnaires/hit_batch_2017_6_14_ambiMASQ_A1PUHCEBSOWETV.csv


In [55]:
data_participants.head(10)

,AID,ASIR,ASIR_had_to_drop_rows,ASIR_unique_responses,BDI,BDI_had_to_drop_rows,BDI_unique_responses,CESD,CESD_had_to_drop_rows,CESD_unique_responses,...,PSWQ_unique_responses,STAI_State,STAI_State_had_to_drop_rows,STAI_State_unique_responses,STAI_Trait,STAI_Trait_anx,STAI_Trait_dep,STAI_Trait_had_to_drop_rows,STAI_Trait_unique_responses,date
0,33LK57MYLUA9QA3R3JQA60A1P97ZST,28,0,5,1,0,2,5,0,4,...,4,37,0,3,42,20,22,0,4,2017-04-17
0,3QRYMNZ7FZMGOL2NPVK6LIZEHEGNTS,58,0,5,11,0,3,19,0,3,...,5,41,0,3,49,15,34,0,3,2017-04-17
0,34YB12FSQZT61YVHTU6Z9KVPMT9MGR,25,0,4,7,0,3,4,0,4,...,4,28,0,4,28,10,18,0,4,2017-04-17
0,3DUZQ9U6SNTSHQYH2M17LUX51XBVSX,10,0,3,3,0,2,6,0,4,...,4,34,0,4,37,9,28,0,3,2017-04-17
0,30LB5CDZNDF9P1JFUH7QWU4IUHT0ZC,30,0,4,15,0,2,22,0,4,...,4,46,0,3,42,11,31,0,4,2017-04-17
0,3H0W84IWBL7ZE0CIS7CSVG4NF1NREZ,92,0,5,14,0,4,12,0,4,...,5,36,0,3,38,16,22,0,4,2017-04-17
0,3PH3VY7DJM22XRK0NO0B3PDMVMGWZI,101,0,5,22,0,4,20,0,3,...,4,26,0,4,51,18,33,0,2,2017-04-17
0,35LDD5557B9B3KR0JLRE7CNITBZKM3,NaN,0,5,37,0,4,45,0,3,...,4,61,0,4,56,18,38,0,4,2017-04-17
0,3P59JYT76MPKZZDU3CMMTUWFUUI2TH,30,0,3,2,0,2,8,0,3,...,3,32,0,3,38,12,26,0,3,2017-04-17
0,3Z9WI9EOZ0TPB6QPSOB1F7FMCPGHK7,22,0,4,1,0,2,9,0,2,...,4,32,0,4,42,10,32,1,3,2017-04-17


### Add Came Back Twice

In [56]:
removeindex = []
for MID in data_participants.MID.unique():
    indices = list(data_participants.loc[data_participants.MID==MID,].index)
    #print(len(indices))
    if indices>1: 
        removeindex = removeindex + indices[1::]

data_participants.loc[removeindex,'CameBackTwice']=1
data_participants.to_csv('../results/sub_table.csv',index=None)
print(sum(data_participants['CameBackTwice']==1))

0


In [57]:
data_participants.to_csv('../results/sub_table_questionnaires_scores.csv',index=None)
data_participants.to_csv('../results/sub_table.csv',index=None)
print(len(data_participants))
data_participants.head()

193


,AID,ASIR,ASIR_had_to_drop_rows,ASIR_unique_responses,BDI,BDI_had_to_drop_rows,BDI_unique_responses,CESD,CESD_had_to_drop_rows,CESD_unique_responses,...,STAI_State,STAI_State_had_to_drop_rows,STAI_State_unique_responses,STAI_Trait,STAI_Trait_anx,STAI_Trait_dep,STAI_Trait_had_to_drop_rows,STAI_Trait_unique_responses,date,CameBackTwice
0,33LK57MYLUA9QA3R3JQA60A1P97ZST,28,0,5,1,0,2,5,0,4,...,37,0,3,42,20,22,0,4,2017-04-17,NaN
0,3QRYMNZ7FZMGOL2NPVK6LIZEHEGNTS,58,0,5,11,0,3,19,0,3,...,41,0,3,49,15,34,0,3,2017-04-17,NaN
0,34YB12FSQZT61YVHTU6Z9KVPMT9MGR,25,0,4,7,0,3,4,0,4,...,28,0,4,28,10,18,0,4,2017-04-17,NaN
0,3DUZQ9U6SNTSHQYH2M17LUX51XBVSX,10,0,3,3,0,2,6,0,4,...,34,0,4,37,9,28,0,3,2017-04-17,NaN
0,30LB5CDZNDF9P1JFUH7QWU4IUHT0ZC,30,0,4,15,0,2,22,0,4,...,46,0,3,42,11,31,0,4,2017-04-17,NaN


### Add Number of Trials

In [58]:
data_participants = pd.read_csv('../results/sub_table.csv')
for index, row in data_participants.iterrows():
    MID = row.MID
    
    datestr = row.date
    datestr = datestr.replace('-','_')
    datestr = datestr.replace('09','9')
    datestr = datestr.replace('06','6')
    datestr = datestr.replace('03','3') ### THIS IS SUPER IMPORTANT
    datestr = datestr.replace('04','4')
    
    for task in ['loss','gain','combined']:
        if task is not 'combined':
            search  = '../results/trial_tables/*'+datestr+'*'+task+'*'+row.MID+'*'
        else:
            search  = '../../tasklogs/combined/*'+datestr+'*'+task+'*'+row.MID+'*'
        files = glob.glob(search)
        if len(files)>0:
            sub = pd.read_csv(files[0]) # load subject data
            
            if task is not 'combined':
                ntrials = np.sum(~np.isnan(sub.mag_u))
                ntrials_responded = np.sum(~np.isnan(sub.choice))
                mean_rt = np.nanmean(sub.rt)
                number_no_resp = np.sum(sub.noresp)
                startingpts = np.nan
                totalpts= np.nan

            else:
                ntrials = np.sum(~np.isnan(sub.mag_left))
                ntrials_responded = np.sum(sub.resp!='999')
                mean_rt = np.nanmean(sub.reaction_time)
                number_no_resp = ntrials-ntrials_responded 
                startingpts = sub.totalpoints.as_matrix()[0]
                totalpts= sub.totalpoints.as_matrix()[-1]
            
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'ntrials_'+task]=ntrials
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'ntrials_resp_'+task]=ntrials_responded
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'mean_RT_'+task]=mean_rt   
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'num_no_resp_'+task]=number_no_resp
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'totalpts'+task]=totalpts
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'startingpts'+task]=startingpts
        else:
            #print(row.MID)
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'ntrials_'+task]=np.nan
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'ntrials_resp_'+task]=np.nan
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'mean_RT_'+task]=np.nan

            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'num_no_resp_'+task]=np.nan  
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'totalpts'+task]=np.nan
            data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                                  'startingpts'+task]=np.nan
            
            
data_participants.to_csv('../results/sub_table.csv',index=None)

### Task Orders

In [59]:
data_participants = pd.read_csv('../results/sub_table.csv')
files = glob.glob('../../batchlogs/*')
f = files[1]
participant_table = pd.read_csv(f) # get table
for f in files:
    participant_table = pd.read_csv(f) # get table
    if 'RPP' in f: # check if its an RPP table
        RPP = 1
    else:
        RPP = 0 
    
    date = str(get_date_from_filename(f))
    
    for MID in participant_table.MID.unique():
        MID = str(MID)
        data_participants.loc[(data_participants['MID']==MID) 
                                  & (data_participants['date']==date),
                              'RPP']=RPP
        data_participants.loc[(data_participants['MID']==MID) 
                                  & (data_participants['date']==date),
                              'tasks']=participant_table.loc[participant_table.MID.astype('str')==MID,'tasks'].as_matrix()[0]
        data_participants.loc[(data_participants['MID']==MID) 
                                  & (data_participants['date']==date),
                              'progress_times']=participant_table.loc[participant_table.MID.astype('str')==MID,'progress_times'].as_matrix()[0]
        
        data_participants.loc[(data_participants['MID']==MID) 
                                  & (data_participants['date']==date),
                              'AID']=participant_table.loc[participant_table.MID.astype('str')==MID,'AID'].as_matrix()[0]
        
        
        
        # for those with 2 tasks # does not save by date... does this cause a problem?/
        if participant_table.loc[participant_table.MID.astype('str')==MID,'tasks'].as_matrix()[0].find('gain')<participant_table.loc[participant_table.MID.astype('str')==MID,'tasks'].as_matrix()[0].find('loss'):
            data_participants.loc[data_participants.MID==MID,'task_order']='gain_first'
        elif participant_table.loc[participant_table.MID.astype('str')==MID,'tasks'].as_matrix()[0].find('gain')>participant_table.loc[participant_table.MID.astype('str')==MID,'tasks'].as_matrix()[0].find('loss'):
            data_participants.loc[data_participants.MID==MID,'task_order']='loss_first'
        
        tasks = participant_table.loc[participant_table.MID.astype('str')==MID,'tasks'].as_matrix()[0]
        if tasks.find('loss')>0 and tasks.find('gain')<0:
            data_participants.loc[data_participants.MID==MID,'task_order']='loss_first'

        if tasks.find('loss')<0 and tasks.find('gain')>0:
            data_participants.loc[data_participants.MID==MID,'task_order']='gain_first'

data_participants.to_csv('../results/sub_table.csv',index=None)

In [60]:
#data_participants.AID

### No Brainer Trials

In [61]:
data_participants = pd.read_csv('../results/sub_table.csv')

In [62]:

### THIS IS UPDATED 3/1/17 - to use. 
    
def calc_no_brainer_per2(MID,date):
    ''' Trys to get a ambi for an MID on a particular date. 
        If they have gain and loss tasks, itll give no brainers for each. 
        If they have combined. Itll give no brainers for gain loss within that'''
    date = date.replace('-','_')
    date = date.replace('09','9')
    date = date.replace('06','6')
    date = date.replace('03','3') ### THIS IS SUPER IMPORTANT
    date = date.replace('04','4')
    
    try:
        ##### LOSS TASK ####### 
        # get loss data 
        files = glob.glob('../../tasklogs/loss/hit*'+date+'*'+MID+'*')
        sub =  get_trial_table(files[0])
        
        # first choose just non-ambiguous trials # 
        sub = sub.loc[(sub['ambig_l']==0) & (sub['ambig_r']==0),]
        sub['mag_left']=sub['mag_left']*-1.0
        sub['mag_right']=sub['mag_right']*-1.0
        left_better = (sub['prob_o_l']<sub['prob_o_r']) & (sub['mag_left']>sub['mag_right'])
        right_better = (sub['prob_o_r']<sub['prob_o_l']) & (sub['mag_right']>sub['mag_left'])
        cor1=(sub.loc[left_better,'resp_r_1']==0).sum() # correctly chose 1
        cor0=(sub.loc[right_better,'resp_r_1']==1).sum() # correctly chose 0 
        n1 = len((sub.loc[left_better,]))
        n0 = len((sub.loc[right_better,]))
        nl=np.float(n1+n0)
        if n1>0:
            perc_correctl = np.float(cor1+cor0)/nl
        else:
            perc_correctl=0.0
    except:
        perc_correctl=np.nan
        nl=np.nan
    
    ##### GAIN TASK #######
    # get gain data 
    try:
        # get gain data 
        files = glob.glob('../../tasklogs/gain/hit*'+date+'*'+MID+'*')
        sub =  get_trial_table(files[0])

        # first choose just non-ambiguous trials # 
        sub = sub.loc[(sub['ambig_l']==0) & (sub['ambig_r']==0),]
        left_better = (sub['prob_o_l']>sub['prob_o_r']) & (sub['mag_left']>sub['mag_right'])
        right_better = (sub['prob_o_r']>sub['prob_o_l']) & (sub['mag_right']>sub['mag_left'])
        cor1=(sub.loc[left_better,'resp_r_1']==0).sum() # correctly chose 1
        cor0=(sub.loc[right_better,'resp_r_1']==1).sum() # correctly chose 0 
        n1 = len((sub.loc[left_better,]))
        n0 = len((sub.loc[right_better,]))
        ng=np.float(n1+n0)
        if ng>0:
            perc_correctg = np.float(cor1+cor0)/ng
        else:
            perc_correctg=0.0

    except:
        perc_correctg=np.nan
        ng=np.nan
        
    ##### Combined Data #######
    try: 
        files = glob.glob('../../tasklogs/combined/*'+date+'*'+MID+'*')
        sub = get_trial_table(files[0],combined=True)

        # first choose just non-ambiguous trials # 
        subnonamb = sub.loc[(sub['ambig_l']==0) & (sub['ambig_r']==0),]

        # then choose gain  # 
        sub = subnonamb.loc[subnonamb['gain_or_loss_trial']=='gain']
        left_better = (sub['prob_o_l']>sub['prob_o_r']) & (sub['mag_left']>sub['mag_right'])
        right_better = (sub['prob_o_r']>sub['prob_o_l']) & (sub['mag_right']>sub['mag_left'])
        cor1=(sub.loc[left_better,'resp_r_1']==0).sum() # correctly chose 1
        cor0=(sub.loc[right_better,'resp_r_1']==1).sum() # correctly chose 0 
        n1 = len((sub.loc[left_better,]))
        n0 = len((sub.loc[right_better,]))
        ng=np.float(n1+n0)
        if ng>0:
            perc_correctg = np.float(cor1+cor0)/ng
        else:
            perc_correctg=0.0
        
        # then choose loss 
        sub = subnonamb.loc[subnonamb['gain_or_loss_trial']=='loss']
        left_better = (sub['prob_o_l']<sub['prob_o_r']) & (sub['mag_left']>sub['mag_right'])
        right_better = (sub['prob_o_r']<sub['prob_o_l']) & (sub['mag_right']>sub['mag_left'])
        cor1=(sub.loc[left_better,'resp_r_1']==0).sum() # correctly chose 1
        cor0=(sub.loc[right_better,'resp_r_1']==1).sum() # correctly chose 0 
        n1 = len((sub.loc[left_better,]))
        n0 = len((sub.loc[right_better,]))
        nl=np.float(n1+n0)
        if n1>0:
            perc_correctl = np.float(cor1+cor0)/nl
        else:
            perc_correctl=0.0
            
    except:
        perc_correctg=np.nan
        ng=np.nan
        perc_correctl=np.nan
        nl=np.nan
    

    return(perc_correctg,perc_correctl,ng,nl)


In [63]:
for index,row in data_participants.iterrows():
    try:
        (corg,corl,ng,nl) = calc_no_brainer_per2(row.MID,row.date)
    except:
       print('failed')
       corg=np.nan
       corl=np.nan
       ng=np.nan
       nl=np.nan
    data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                           'no_brainer_per_cor_gain']=corg
    data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                           'no_brainer_per_cor_loss']=corl
    data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                           'no_brainer_n_gain']=ng
    data_participants.loc[(data_participants['MID']==row.MID) 
                                  & (data_participants['date']==row.date),
                           'no_brainer_n_loss']=nl


In [64]:

data_participants.to_csv('../results/sub_table.csv',index=None)

### Timing of Surveys

In [65]:
data_participants = pd.read_csv('../results/sub_table.csv')
files = glob.glob('../../batchlogs/*')
for f in files:

    d =pd.read_csv(f) # get table

    # loop through this table
    for index, row in d.iterrows():
        
        MID = str(row.MID)
        try:
            #print(row)
            progress_times = row.progress_times.split(';') # find corresponding start and end time
            
            
            # Questionnaires
            for name in ['STAI_State','STAI_Trait','BDI','MASQ','EPQ','CESD','PSWQ','ASIR','IUS']:
                matching = [s for s in progress_times if name+" start" in s]
                if matching is not None:
                    start = matching[0].split(':')[1]
                    matching = [s for s in progress_times if name+" end" in s]
                    end = matching[0].split(':')[1]
                    duration = float(end)-float(start)
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'duration_'+name]=duration
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'starttime_'+name]=float(start)
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'endtime_'+name]=float(end)
                else:
                    data_participants.loc[(data_participants.MID==MID)
                                          & (data_participants.date==str(get_date_from_filename(f))),'duration_'+name]=np.nan
                    data_participants.loc[(data_participants.MID==MID)
                                          & (data_participants.date==str(get_date_from_filename(f))),'starttime_'+name]=np.nan
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'endtime_'+name]=np.nan

        except: 
            print('failed')
            print(f)
            #print(MID)

data_participants.to_csv('../results/sub_table.csv',index=None)

failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_10_10_RPP_ambiparticipant_table.csv
failed
../../batchlogs/hit_batch_2016_

limit_output extension: Maximum message size of 10000 exceeded with 10027 characters

### Timing of Tasks

In [66]:
data_participants = pd.read_csv('../results/sub_table.csv')
for f in files:

    d =pd.read_csv(f) # get table

    # loop through this table
    for index, row in d.iterrows():
        
        MID = str(row.MID)
        try:
            #print(row)
            progress_times = row.progress_times2.split(';') # find corresponding start and end time
            
            
            # Questionnaires
            for name in ['ambi combined']:
                matching = [s for s in progress_times if name+" start" in s]
                if matching is not None:
                    #start = datetime.datetime.strptime(matching[0].split(' ')[-1].split('.')[0],'%H:%M:%S')
                    start = datetime.datetime.strptime(matching[0].split('start')[-1],':%Y-%m-%d %H:%M:%S.%f')
                    start_abs_seconds = (start-datetime.datetime(1970,1,1)).total_seconds()

                    matching = [s for s in progress_times if name+" end" in s]
                    #end = datetime.datetime.strptime(matching[0].split(' ')[-1].split('.')[0],'%H:%M:%S')
                    # puts date in
                    end = datetime.datetime.strptime(matching[0].split('end')[-1],':%Y-%m-%d %H:%M:%S.%f')
                    end_abs_seconds = (end-datetime.datetime(1970,1,1)).total_seconds()
                    
                    duration = end-start
                    #print(duration.seconds)
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'duration_'+name]=float(duration.seconds)
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'starttime_'+name]=float(start_abs_seconds)
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'endtime_'+name]=float(end_abs_seconds)
                else:
                    data_participants.loc[(data_participants.MID==MID)
                                          & (data_participants.date==str(get_date_from_filename(f))),'duration_'+name]=np.nan
                    data_participants.loc[(data_participants.MID==MID)
                                          & (data_participants.date==str(get_date_from_filename(f))),'starttime_'+name]=np.nan
                    data_participants.loc[(data_participants.MID==MID) 
                                          & (data_participants.date==str(get_date_from_filename(f))),'endtime_'+name]=np.nan

        except: 
            x=1

data_participants.to_csv('../results/sub_table.csv',index=None)

In [67]:
data_participants.iloc[-1]

AID                        3D8YOU6S9FPNHDBKQJOZO3ZU9436U5
ASIR                                                  NaN
ASIR_had_to_drop_rows                                 NaN
ASIR_unique_responses                                 NaN
BDI                                                    31
BDI_had_to_drop_rows                                    0
BDI_unique_responses                                    4
CESD                                                   26
CESD_had_to_drop_rows                                   1
CESD_unique_responses                                   3
EPQ                                                    22
EPQ_had_to_drop_rows                                    0
EPQ_unique_responses                                    2
IUS                                                    93
IUS1                                                   44
IUS2                                                   49
IUS_had_to_drop_rows                                    0
IUS_unique_res

### Adding parameters from logistic regression

In [71]:
data_participants = pd.read_csv('../results/sub_table.csv')

results_folder = '../results/log_reg_model_results/individual_subjects/'
resultnames = ['bic','aic','pseudoR2','pred_acc']

whichones = ['0','1','2']
whichones = ['3','4']
#whichones = ['5','6','7','8','9']
whichones = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25']
# whichones = ['10','11','13','14']

# whichones = ['0','1','8']
# whichones = ['-1']
# whichones = ['2','3']

#whichones = ['-1','2','3','4','5']
whichones=['4','5','6']
whichones=['18']
for paramnum in whichones:
    try:
        
        for split in ['ambig_gain','ambig_loss']:
        #for split in ['unambig_gain','unambig_loss']:
            modelname = 'model_split_'+split+'_'.join(param_sets_split[paramnum])
            files = glob.glob(results_folder+'*'+modelname+'.p')
            for filee in files:
                model_results = pickle.load(open(filee,'rb'))
                MID = str(model_results['MID'])
                task = get_gain_loss_from_filename(filee)
                params = model_results['params']

                for result in resultnames:
                    data_participants.loc[(data_participants.MID==MID) & 
                                      (data_participants.date==str(get_date_from_filename(filee))),
                                       modelname+'_'+result+'_'+task]=model_results[result]
                for param in params.index:
                    data_participants.loc[(data_participants.MID==MID) & 
                                      (data_participants.date==str(get_date_from_filename(filee))),
                                       modelname+'_'+param+'_'+task]=params[param]
    except:
        print('didnt add:'+paramnum)
        
data_participants.to_csv('../results/sub_table.csv',index=None)

### Mscl

In [79]:
data_participants = pd.read_csv('../results/sub_table.csv')
data_participants.MID = data_participants.MID.astype('str')
data_participants.to_csv('../results/sub_table.csv',index=None)

## Load Data (Examples)

### Load Data (Specify Group of Subjects )

In [102]:

data_participants = pd.read_csv('../results/sub_table.csv')
data_participants = data_participants.loc[
    ~np.isnan(data_participants.num_no_resp_combined)&
    ((data_participants.date=='2017-04-17')|(data_participants.date=='2017-03-06'))|(data_participants.date=='2017-06-14')]
datasetfolder='datasetb'
data_participants.head()

,AID,ASIR,ASIR_had_to_drop_rows,ASIR_unique_responses,BDI,BDI_had_to_drop_rows,BDI_unique_responses,CESD,CESD_had_to_drop_rows,CESD_unique_responses,...,model_split_ambig_lossmag_diff_prob_diff_sqrt_prop_revealed_prob_total_inter_prob_diff_sqrt_prop_revealed_inter_prob_total_sqrt_prop_revealed_inter_prob_total_prob_diff_sqrt_prop_revealed_inter_prob_total_prob_diff_sqrt_prop_revealed_combined,flexible_prior0_bic_combined,flexible_prior0_aic_combined,flexible_prior0_pseudoR2_combined,flexible_prior0_pred_acc_combined,flexible_prior0_B0_combined,flexible_prior0_B1_combined,flexible_prior0_B2_combined,flexible_prior0_alpha_combined,flexible_prior0_beta_combined
0,33LK57MYLUA9QA3R3JQA60A1P97ZST,28,0,5,1,0,2,5,0,4,...,NaN,78.898181,67.114637,0.439177,0.897436,-0.651968,-2.589137,0.068135,20.000000,20.000000
1,3QRYMNZ7FZMGOL2NPVK6LIZEHEGNTS,58,0,5,11,0,3,19,0,3,...,NaN,NaN,NaN,NaN,0.564103,-10.000000,10.000000,7.159974,5.436703,16.828380
2,34YB12FSQZT61YVHTU6Z9KVPMT9MGR,25,0,4,7,0,3,4,0,4,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
3,3DUZQ9U6SNTSHQYH2M17LUX51XBVSX,10,0,3,3,0,2,6,0,4,...,NaN,107.894537,96.110993,0.189753,0.705128,-0.024690,5.437591,0.011709,4.593528,5.931948
4,30LB5CDZNDF9P1JFUH7QWU4IUHT0ZC,30,0,4,15,0,2,22,0,4,...,NaN,NaN,NaN,NaN,0.756410,-0.534918,1.909465,10.000000,11.628110,12.987043


In [103]:
data_participants.loc[data_participants.MID=='A1GHQFNWXYP1RS']

,AID,ASIR,ASIR_had_to_drop_rows,ASIR_unique_responses,BDI,BDI_had_to_drop_rows,BDI_unique_responses,CESD,CESD_had_to_drop_rows,CESD_unique_responses,...,model_split_ambig_lossmag_diff_prob_diff_sqrt_prop_revealed_prob_total_inter_prob_diff_sqrt_prop_revealed_inter_prob_total_sqrt_prop_revealed_inter_prob_total_prob_diff_sqrt_prop_revealed_inter_prob_total_prob_diff_sqrt_prop_revealed_combined,flexible_prior0_bic_combined,flexible_prior0_aic_combined,flexible_prior0_pseudoR2_combined,flexible_prior0_pred_acc_combined,flexible_prior0_B0_combined,flexible_prior0_B1_combined,flexible_prior0_B2_combined,flexible_prior0_alpha_combined,flexible_prior0_beta_combined
192,3D8YOU6S9FPNHDBKQJOZO3ZU9436U5,NaN,NaN,NaN,31,0,4,26,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
data_participants.date.unique()

array(['2017-04-17', '2017-06-14'], dtype=object)

In [ ]:
#data_participants['model_split_amb_unamb_gain_loss_aic_combined']

### Load single model data w parameter and beta as columns

In [ ]:
modelnames

In [ ]:
modelname = 'model_singRL_mag_diff_rl_prob_diff_rl_ambig_present_sqrt_prop_revealed'
modelname = 'model_split_amb_unamb_gain_loss'
df=get_params_df(modelname,'combined',data_participants,combined=True)
df.head()